Single Race Exploratory Data Analysis -

The purpose of this notebook is to explore and wrangle data from the Fast-F1 API for a single race. This serves as a starting point to establish a clean workflow for data acquisition, preparation, and early exploration before scaling to a multi-race analysis. I will focus on building a data dictionary, assessing data quality, generating descriptive statistics, and creating preliminary visualizations to understand the structure and sufficiency of the data. The outcome of this notebook will be a reproducible workflow for future feature engineering and multi-race EDA.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [1]:
import sys
import os
import logging

# Add the root directory to sys.path
root = os.path.abspath("..")
sys.path.append(root)

# Suppress FastF1 info logs globally
logging.getLogger('fastf1').setLevel(logging.WARNING)

In this section, I import Python libraries for data visualization, numerical analysis, and working with Pandas dataframes that the FastF1 API is primarily structured with. I also import custom modules for accessing preprocessed F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [2]:
from src.data import f1_data
from src.utils import f1_constants, f1_pandas_helpers
from src.preprocessing import telemetry_cleaning

from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Set pandas display options
pd.set_option('display.max_rows', None)  # reset_option to compact view
pd.set_option('display.max_columns', None)


The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1data.py), which creates a session object built on top of FastF1. This object provides access to key race data as well as custom functions I’ve implemented.

The session parameters were chosen to best match Tier 1 control qualities:

- Weather: Abu Dhabi (dry conditions)
- Max Speed: C5 Ultra Soft Tires
- Minimize Outliers: Top 3 valid laps from Q3
- Traffic: Remove tow laps to avoid slipstream bias

In [3]:
# Define session parameters
year = 2024
grand_prix = f1_constants.F1Constants.LOCATIONS["Abu Dhabi"]
session_type = f1_constants.F1Constants.SESSIONS["Q"]

# Call session object
session = f1_data.F1Session(year, grand_prix, session_type)

req         WARNING 	DEFAULT CACHE ENABLED! (671.13 MB) /Users/phillipsmith/Library/Caches/fastf1
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '55', '27', '1', '10', '63', '14', '77', '11', '22', '30', '18', '16', '20', '23', '24', '44', '43', '61']


All drivers who participated in the Q2 and Q3 sessions will be analyzed and assigned variables to be identified by their three-letter name code.

In [4]:
# Drivers from each team that participated in Q2 and Q3 accessed by their three-letter code
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
perez = f1_constants.F1Constants.DRIVERS["Sergio Perez"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
bottas = f1_constants.F1Constants.DRIVERS["Valtteri Bottas"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]
magnussen = f1_constants.F1Constants.DRIVERS["Kevin Magnussen"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]

Q3 data will be used for each driver whenever available; otherwise, Q2 data will be selected. For each driver, their fastest qualifying lap is identified by filtering timestamps for each sector. The resulting sector datasets are cleaned and then merged with other drivers’ telemetry data for that sector.

In [ ]:
# Q3 data for Lando Norris, McLaren
q1, q2, q3 = session.get_laps(norris).split_qualifying_sessions()
norris_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:59.496000"
sector_1_end = "0 days 01:17:16.260000"
sector_2_end = "0 days 01:17:52.036000"
sector_3_end = "0 days 01:18:21.897000"

norris_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_start, end=sector_1_end)
norris_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector1_telemetry, norris)

# find_steering_wheel_angle()

norris_sector1_telemetry_cleaned

# norris_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_end, end=sector_2_end)
# norris_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector2_telemetry, norris)
# norris_sector2_telemetry_cleaned

# norris_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_2_end, end=sector_3_end)
# norris_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector3_telemetry, norris)
# norris_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7979,NOR,01:16:59.557,00:17:19.828,11229.000000,68.333333,6,100.000000,False,20968.508889,778.569904,2105.751713,-239.001361
7980,NOR,01:16:59.654,00:17:19.925,11318.724941,68.932098,6,100.000000,False,20975.275273,844.000000,2113.000000,-239.000000
7981,NOR,01:16:59.874,00:17:20.145,11522.224941,70.290123,6,100.000000,False,20990.849977,997.000000,2130.000000,-239.000000
7982,NOR,01:16:59.917,00:17:20.188,11562.000000,70.555556,6,100.000000,False,20993.908889,1026.891384,2133.312099,-238.974188
7983,NOR,01:17:00.055,00:17:20.326,11665.883237,70.981481,6,100.000000,False,21003.748877,1123.000000,2144.000000,-239.000000
7984,NOR,01:17:00.274,00:17:20.545,11830.741618,71.657407,6,100.000000,False,21019.491494,1280.000000,2162.000000,-240.000000
7985,NOR,01:17:00.277,00:17:20.548,11833.000000,71.666667,6,100.000000,False,21019.708889,1282.167908,2162.253093,-240.006256
7986,NOR,01:17:00.474,00:17:20.745,11091.172116,72.692708,6,100.000000,False,21034.089617,1425.000000,2179.000000,-240.000000
7987,NOR,01:17:00.597,00:17:20.868,10628.000000,73.333333,7,100.000000,False,21043.175556,1514.747481,2189.147597,-240.558748
7988,NOR,01:17:00.693,00:17:20.964,10657.866667,73.777778,7,100.000000,False,21050.324912,1585.000000,2197.000000,-241.000000


In [8]:
# Q3 data for Oscar Piastri, McLaren
q1, q2, q3 = session.get_laps(piastri).split_qualifying_sessions()
piastri_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:27.506000"
sector_1_end = "0 days 01:16:44.333000"
sector_2_end = "0 days 01:17:20.186000"
sector_3_end = "0 days 01:17:50.110000"

piastri_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_start, end=sector_1_end)
piastri_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector1_telemetry, piastri)
piastri_sector1_telemetry_cleaned

# piastri_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_end, end=sector_2_end)
# piastri_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector2_telemetry, piastri)
# piastri_sector2_telemetry_cleaned

# piastri_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_2_end, end=sector_3_end)
# piastri_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector3_telemetry, piastri)
# piastri_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7757,PIA,01:16:27.517,00:16:51.262,11223.000000,67.777778,6,99.000000,False,20977.372778,747.295487,2102.262937,-239.348242
7758,PIA,01:16:27.614,00:16:51.359,11327.759931,68.316666,6,99.000000,False,20984.024050,832.000000,2112.000000,-239.000000
7759,PIA,01:16:27.717,00:16:51.462,11439.000000,68.888889,6,99.000000,False,20991.150556,967.893580,2127.202551,-238.977293
7760,PIA,01:16:27.773,00:16:51.518,11462.100000,69.277778,6,99.000000,False,20995.062533,1038.000000,2135.000000,-239.000000
7761,PIA,01:16:27.957,00:16:51.702,11538.000000,70.555556,6,99.000000,False,21008.083889,1172.574185,2149.922892,-238.873758
7762,PIA,01:16:28.234,00:16:51.979,11124.808429,71.677662,6,99.000000,False,21028.070399,1264.000000,2160.000000,-239.000000
7763,PIA,01:16:28.437,00:16:52.182,10822.000000,72.500000,6,99.000000,False,21042.883889,1409.271712,2176.465552,-239.967514
7764,PIA,01:16:28.454,00:16:52.199,10787.260987,72.587991,6,99.000000,False,21044.127198,1422.000000,2178.000000,-240.000000
7765,PIA,01:16:28.554,00:16:52.299,10582.913079,73.105590,6,99.000000,False,21051.452482,1495.000000,2187.000000,-240.000000
7766,PIA,01:16:28.598,00:16:52.343,10493.000000,73.333333,7,99.000000,False,21054.690556,1525.873129,2190.589715,-240.006482


In [9]:
# Q3 data for Max Verstappen, Red Bull
q1, q2, q3 = session.get_laps(verstappen).split_qualifying_sessions()
verstappen_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:08:27.031000"
sector_1_end = "0 days 01:08:44.036000"
sector_2_end = "0 days 01:09:19.766000"
sector_3_end = "0 days 01:09:49.959000"

verstappen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_start, end=sector_1_end)
verstappen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector1_telemetry, verstappen)
verstappen_sector1_telemetry_cleaned

# verstappen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_end, end=sector_2_end)
# verstappen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector2_telemetry, verstappen)
# verstappen_sector2_telemetry_cleaned

# verstappen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_2_end, end=sector_3_end)
# verstappen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector3_telemetry, verstappen)
# verstappen_sector3_telemetry_cleaned


,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8720,VER,01:08:27.035,00:19:22.332,10528.000000,66.388889,6,100.000000,False,5283.763056,620.107599,2086.474071,-238.993586
8721,VER,01:08:27.213,00:19:22.510,10833.381030,67.315972,6,100.000000,False,5295.798162,740.000000,2101.000000,-239.000000
8722,VER,01:08:27.355,00:19:22.652,11077.000000,68.055556,6,100.000000,False,5305.540833,836.153523,2112.077254,-239.000272
8723,VER,01:08:27.393,00:19:22.690,11103.124912,68.319444,6,100.000000,False,5308.168325,862.000000,2115.000000,-239.000000
8724,VER,01:08:27.595,00:19:22.892,11242.000000,69.722222,6,100.000000,False,5322.274167,1000.712662,2130.423372,-239.000297
8725,VER,01:08:27.654,00:19:22.951,11242.000000,70.049999,6,100.000000,False,5326.434899,1042.000000,2135.000000,-239.000000
8726,VER,01:08:27.795,00:19:23.092,11242.000000,70.833333,6,100.000000,False,5336.440833,1142.344379,2146.173284,-238.997784
8727,VER,01:08:27.894,00:19:23.191,11388.024717,71.177083,6,100.000000,False,5343.503583,1211.000000,2154.000000,-239.000000
8728,VER,01:08:28.035,00:19:23.332,11596.000000,71.666667,6,100.000000,False,5353.640833,1306.961891,2165.293984,-239.007390
8729,VER,01:08:28.073,00:19:23.370,11522.443105,71.855158,6,100.000000,False,5356.395605,1339.000000,2169.000000,-239.000000


In [10]:
# Q3 data for Sergio Perez, Red Bull
q1, q2, q3 = session.get_laps(perez).split_qualifying_sessions()
perez_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:14:50.040000"
sector_1_end = "0 days 01:15:06.945000"
sector_2_end = "0 days 01:15:42.890000"
sector_3_end = "0 days 01:16:13.127000"

perez_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_start, end=sector_1_end)
perez_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector1_telemetry, perez)
perez_sector1_telemetry_cleaned

# perez_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_end, end=sector_2_end)
# perez_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector2_telemetry, perez)
# perez_sector2_telemetry_cleaned

# perez_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_2_end, end=sector_3_end)
# perez_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector3_telemetry, perez)
# perez_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
6955,PER,01:14:50.153,00:15:08.450,10903.550000,67.375000,6,100.000000,False,20999.805644,811.000000,2109.000000,-240.000000
6956,PER,01:14:50.237,00:15:08.534,10962.000000,68.055556,6,100.000000,False,21005.555833,905.500777,2119.680453,-239.630629
6957,PER,01:14:50.354,00:15:08.651,11060.279946,68.543055,6,100.000000,False,21013.598746,1042.000000,2135.000000,-239.000000
6958,PER,01:14:50.437,00:15:08.734,11130.000000,68.888889,6,100.000000,False,21019.333611,1111.283557,2142.747413,-238.712846
6959,PER,01:14:50.637,00:15:08.934,11217.000000,69.722222,6,100.000000,False,21033.278056,1189.138168,2151.351162,-238.577211
6960,PER,01:14:50.814,00:15:09.111,11422.319926,70.705555,6,100.000000,False,21045.808865,1264.000000,2160.000000,-239.000000
6961,PER,01:14:50.837,00:15:09.134,11449.000000,70.833333,6,100.000000,False,21047.444722,1277.442413,2161.579139,-239.088995
6962,PER,01:14:50.997,00:15:09.294,11530.000000,71.388889,6,100.000000,False,21058.866944,1391.201914,2175.012403,-239.838257
6963,PER,01:14:51.054,00:15:09.351,11549.542835,71.615079,6,100.000000,False,21062.968095,1434.000000,2180.000000,-240.000000
6964,PER,01:14:51.273,00:15:09.570,11624.628571,72.484127,6,100.000000,False,21078.874940,1592.000000,2198.000000,-240.000000


In [11]:
# Q3 data for Carlos Sainz, Ferrari
q1, q2, q3 = session.get_laps(sainz).split_qualifying_sessions()
sainz_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:53.219000"
sector_1_end = "0 days 01:17:10.014000"
sector_2_end = "0 days 01:17:45.801000"
sector_3_end = "0 days 01:18:15.858000"

sainz_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_start, end=sector_1_end)
sainz_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector1_telemetry, sainz)
sainz_sector1_telemetry_cleaned

# sainz_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_end, end=sector_2_end)
# sainz_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector2_telemetry, sainz)
# sainz_sector2_telemetry_cleaned

# sainz_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_2_end, end=sector_3_end)
# sainz_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector3_telemetry, sainz)
# sainz_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7558,SAI,01:16:53.237,00:16:22.603,11272.000000,67.777778,6,100.000000,False,21018.141667,785.538510,2106.896035,-238.930622
7559,SAI,01:16:53.454,00:16:22.820,11389.799965,68.423611,6,100.000000,False,21032.981595,871.000000,2116.000000,-239.000000
7560,SAI,01:16:53.517,00:16:22.883,11424.000000,68.611111,6,100.000000,False,21037.352778,908.296727,2120.074619,-239.006136
7561,SAI,01:16:53.714,00:16:23.080,11595.280500,69.979166,6,100.000000,False,21051.269773,1052.000000,2136.000000,-239.000000
7562,SAI,01:16:53.877,00:16:23.243,11737.000000,71.111111,6,100.000000,False,21062.952778,1173.156586,2149.625102,-239.004775
7563,SAI,01:16:53.894,00:16:23.260,11691.596004,71.189815,6,100.000000,False,21064.174832,1185.000000,2151.000000,-239.000000
7564,SAI,01:16:54.094,00:16:23.460,11157.429338,72.115740,6,100.000000,False,21078.614627,1328.000000,2168.000000,-239.000000
7565,SAI,01:16:54.117,00:16:23.483,11096.000000,72.222222,6,100.000000,False,21080.286111,1350.371010,2170.557586,-239.053809
7566,SAI,01:16:54.277,00:16:23.643,10816.000000,73.055556,7,100.000000,False,21091.975000,1539.613354,2191.620197,-239.762016
7567,SAI,01:16:54.334,00:16:23.700,10758.050065,73.187500,7,100.000000,False,21096.153757,1597.000000,2198.000000,-240.000000


In [12]:
# Q2 data for Charles Leclerc, Ferrari
q1, q2, q3 = session.get_laps(leclerc).split_qualifying_sessions()
leclerc_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:03.473000"
sector_1_end = "0 days 00:57:20.332000"
sector_2_end = "0 days 00:57:56.296000"
sector_3_end = "0 days 00:58:26.338000"

leclerc_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_start, end=sector_1_end)
leclerc_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector1_telemetry, leclerc)
leclerc_sector1_telemetry_cleaned

# leclerc_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_end, end=sector_2_end)
# leclerc_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector2_telemetry, leclerc)
# leclerc_sector2_telemetry_cleaned

# leclerc_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_2_end, end=sector_3_end)
# leclerc_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector3_telemetry, leclerc)
# leclerc_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
9966,LEC,00:57:03.493,00:22:04.642,11163.500000,67.638889,6,99.500000,False,20984.786415,720.000000,2099.000000,-239.000000
9967,LEC,00:57:03.673,00:22:04.822,11316.000000,68.611111,6,100.000000,False,20997.214444,887.665451,2118.181363,-238.988591
9968,LEC,00:57:03.753,00:22:04.902,11378.000000,68.849206,6,100.000000,False,21002.745420,966.000000,2127.000000,-239.000000
9969,LEC,00:57:03.953,00:22:05.102,11533.000000,69.444444,6,100.000000,False,21016.658889,1133.454154,2145.843524,-239.023025
9970,LEC,00:57:04.153,00:22:05.302,11653.000000,70.833333,6,100.000000,False,21030.825556,1274.696003,2161.787223,-239.011466
9971,LEC,00:57:04.194,00:22:05.343,11687.472594,70.946655,6,100.000000,False,21033.740793,1303.000000,2165.000000,-239.000000
9972,LEC,00:57:04.354,00:22:05.503,11822.000000,71.388889,6,100.000000,False,21045.174722,1411.462207,2177.371060,-238.925418
9973,LEC,00:57:04.574,00:22:05.723,10781.583333,72.916667,6,100.000000,False,21061.236883,1548.000000,2193.000000,-239.000000
9974,LEC,00:57:04.594,00:22:05.743,10687.000000,73.055556,6,100.000000,False,21062.708056,1559.145611,2194.272026,-239.035615
9975,LEC,00:57:04.794,00:22:05.943,10272.000000,73.888889,7,100.000000,False,21077.485833,1659.616016,2205.690918,-239.648192


In [13]:
# Q3 data for Valtteri Bottas, Alfa Romeo
q1, q2, q3 = session.get_laps(bottas).split_qualifying_sessions()
bottas_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:15:46.639000"
sector_1_end = "0 days 01:16:03.650000"
sector_2_end = "0 days 01:16:39.552000"
sector_3_end = "0 days 01:17:09.702000"

bottas_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_start, end=sector_1_end)
bottas_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector1_telemetry, bottas)
bottas_sector1_telemetry_cleaned

# bottas_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_end, end=sector_2_end)
# bottas_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector2_telemetry, bottas)
# bottas_sector2_telemetry_cleaned

# bottas_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_2_end, end=sector_3_end)
# bottas_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector3_telemetry, bottas)
# bottas_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
6743,BOT,01:15:46.674,00:14:35.069,11214.452742,67.450617,6,100.000000,False,5310.733762,781.000000,2106.000000,-239.000000
6744,BOT,01:15:46.914,00:14:35.309,11347.119409,68.932098,6,100.000000,False,5327.373757,904.000000,2120.000000,-239.000000
6745,BOT,01:15:46.997,00:14:35.392,11393.000000,69.444444,6,100.000000,False,5333.194722,954.014230,2125.771893,-239.440098
6746,BOT,01:15:47.274,00:14:35.669,11662.085652,70.818452,6,100.000000,False,5352.815141,1155.000000,2148.000000,-240.000000
6747,BOT,01:15:47.277,00:14:35.672,11665.000000,70.833333,6,100.000000,False,5353.028056,1157.117896,2148.203795,-239.955296
6748,BOT,01:15:47.334,00:14:35.729,11717.724941,70.932292,6,100.000000,False,5357.072793,1197.000000,2152.000000,-239.000000
6749,BOT,01:15:47.437,00:14:35.832,11813.000000,71.111111,6,100.000000,False,5364.405833,1269.647817,2160.056163,-238.437606
6750,BOT,01:15:47.574,00:14:35.969,11333.500224,72.062500,6,100.000000,False,5374.288089,1367.000000,2172.000000,-239.000000
6751,BOT,01:15:47.597,00:14:35.992,11253.000000,72.222222,6,100.000000,False,5375.961389,1383.285688,2173.926982,-239.042176
6752,BOT,01:15:47.854,00:14:36.249,10941.846506,73.497023,6,100.000000,False,5394.874961,1570.000000,2195.000000,-239.000000


In [14]:
# Q3 data for Fernando Alonso, Aston Martin
q1, q2, q3 = session.get_laps(alonso).split_qualifying_sessions()
alonso_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:45.371000"
sector_1_end = "0 days 01:17:02.191000"
sector_2_end = "0 days 01:17:38.197000"
sector_3_end = "0 days 01:18:08.377000"

alonso_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_start, end=sector_1_end)
alonso_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector1_telemetry, alonso)
alonso_sector1_telemetry_cleaned

# alonso_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_end, end=sector_2_end)
# alonso_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector2_telemetry, alonso)
# alonso_sector2_telemetry_cleaned

# alonso_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_2_end, end=sector_3_end)
# alonso_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector3_telemetry, alonso)
# alonso_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7644,ALO,01:16:45.534,00:16:36.068,11150.903282,68.670139,6,99.000000,False,20995.761209,866.000000,2116.000000,-239.000000
7645,ALO,01:16:45.677,00:16:36.211,10801.000000,69.166667,6,99.000000,False,21005.726389,1030.313935,2134.173839,-239.004220
7646,ALO,01:16:45.694,00:16:36.228,10779.466748,69.304398,6,99.000000,False,21006.921261,1047.000000,2136.000000,-239.000000
7647,ALO,01:16:45.917,00:16:36.451,10497.000000,71.111111,7,99.000000,False,21022.793056,1153.318499,2147.304022,-238.837367
7648,ALO,01:16:46.054,00:16:36.588,10594.612454,71.824652,7,99.000000,False,21032.720069,1231.000000,2156.000000,-239.000000
7649,ALO,01:16:46.237,00:16:36.771,10725.000000,72.777778,7,99.000000,False,21046.081944,1434.195700,2179.445832,-239.770282
7650,ALO,01:16:46.294,00:16:36.828,10768.937451,72.843750,7,99.000000,False,21050.236420,1491.000000,2186.000000,-240.000000
7651,ALO,01:16:46.477,00:16:37.011,10910.000000,73.055556,7,99.000000,False,21063.615278,1600.794247,2198.521539,-240.461956
7652,ALO,01:16:46.757,00:16:37.291,10964.000000,74.722222,7,99.000000,False,21084.537500,1737.961625,2213.838602,-240.085875
7653,ALO,01:16:46.775,00:16:37.309,10985.824845,74.815972,7,99.000000,False,21085.894887,1757.000000,2216.000000,-240.000000


In [15]:
# Q2 data for Lance Stroll, Aston Martin
q1, q2, q3 = session.get_laps(stroll).split_qualifying_sessions()
stroll_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:56:38.573000"
sector_1_end = "0 days 00:56:55.483000"
sector_2_end = "0 days 00:57:31.468000"
sector_3_end = "0 days 00:58:02.147000"

stroll_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_start, end=sector_1_end)
stroll_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector1_telemetry, stroll)
stroll_sector1_telemetry_cleaned

# stroll_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_end, end=sector_2_end)
# stroll_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector2_telemetry, stroll)
# stroll_sector2_telemetry_cleaned

# stroll_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_2_end, end=sector_3_end)
# stroll_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector3_telemetry, stroll)
# stroll_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
9912,STR,00:56:38.594,00:21:57.213,11563.000000,67.500000,6,99.000000,False,20975.604444,813.000000,2110.000000,-240.000000
9913,STR,00:56:38.874,00:21:57.493,11573.500000,69.201389,6,99.000000,False,20995.014925,962.000000,2126.000000,-239.000000
9914,STR,00:56:38.914,00:21:57.533,11575.000000,69.444444,6,99.000000,False,20997.826667,987.354971,2128.788597,-238.940062
9915,STR,00:56:39.053,00:21:57.672,11392.562416,70.168403,6,99.000000,False,21007.659243,1087.000000,2140.000000,-239.000000
9916,STR,00:56:39.234,00:21:57.853,11155.000000,71.111111,7,99.000000,False,21020.582222,1213.910394,2154.401920,-239.015380
9917,STR,00:56:39.254,00:21:57.873,11104.700000,71.194444,7,99.000000,False,21022.016375,1228.000000,2156.000000,-239.000000
9918,STR,00:56:39.434,00:21:58.053,10652.000000,71.944444,7,99.000000,False,21034.971111,1356.788153,2170.583789,-238.759257
9919,STR,00:56:39.514,00:21:58.133,10714.800000,72.166667,7,99.000000,False,21040.746719,1413.000000,2177.000000,-239.000000
9920,STR,00:56:39.634,00:21:58.253,10809.000000,72.500000,7,99.000000,False,21049.471111,1494.542894,2186.415859,-240.060054
9921,STR,00:56:39.754,00:21:58.373,10891.500000,73.541667,7,99.000000,False,21058.330459,1587.000000,2197.000000,-241.000000


In [16]:
# Q3 data for Pierre Gasly, Alpine
q1, q2, q3 = session.get_laps(gasly).split_qualifying_sessions()
gasly_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:18.040000"
sector_1_end = "0 days 01:16:34.989000"
sector_2_end = "0 days 01:17:10.860000"
sector_3_end = "0 days 01:17:40.892000"

gasly_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_start, end=sector_1_end)
gasly_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector1_telemetry, gasly)
gasly_sector1_telemetry_cleaned

# gasly_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_end, end=sector_2_end)
# gasly_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector2_telemetry, gasly)
# gasly_sector2_telemetry_cleaned

# gasly_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_2_end, end=sector_3_end)
# gasly_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector3_telemetry, gasly)
# gasly_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7829,GAS,01:16:18.234,00:17:01.094,11068.374944,68.312500,6,100.000000,False,21006.656855,863.000000,2115.000000,-239.000000
7830,GAS,01:16:18.357,00:17:01.217,11176.000000,69.166667,6,100.000000,False,21015.232500,935.525033,2123.326468,-239.003087
7831,GAS,01:16:18.474,00:17:01.334,11288.612438,69.708333,6,100.000000,False,21023.431380,1028.000000,2134.000000,-239.000000
7832,GAS,01:16:18.597,00:17:01.457,11407.000000,70.277778,6,100.000000,False,21032.099167,1152.416181,2148.056510,-238.986154
7833,GAS,01:16:18.797,00:17:01.657,11555.000000,71.111111,6,100.000000,False,21046.321389,1355.195843,2170.684501,-238.992904
7834,GAS,01:16:18.814,00:17:01.674,11555.000000,71.199652,6,100.000000,False,21047.540175,1367.000000,2172.000000,-239.000000
7835,GAS,01:16:18.957,00:17:01.817,11555.000000,71.944444,6,100.000000,False,21057.832500,1429.665956,2178.975475,-239.102746
7836,GAS,01:16:19.114,00:17:01.974,11743.399923,71.944444,6,100.000000,False,21069.126196,1483.000000,2185.000000,-240.000000
7837,GAS,01:16:19.117,00:17:01.977,11747.000000,71.944444,6,100.000000,False,21069.343611,1484.526844,2185.173802,-240.032864
7838,GAS,01:16:19.317,00:17:02.177,10767.000000,73.333333,6,100.000000,False,21084.010278,1625.798878,2201.332538,-243.155031


In [17]:
# Q3 data for Nico Hulkenberg, Haas
q1, q2, q3 = session.get_laps(hulkenberg).split_qualifying_sessions()
hulkenberg_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:03.696000"
sector_1_end = "0 days 01:16:20.651000"
sector_2_end = "0 days 01:16:56.369000"
sector_3_end = "0 days 01:17:26.431000"

hulkenberg_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_start, end=sector_1_end)
hulkenberg_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector1_telemetry, hulkenberg)
hulkenberg_sector1_telemetry_cleaned

# hulkenberg_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_end, end=sector_2_end)
# hulkenberg_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector2_telemetry, hulkenberg)
# hulkenberg_sector2_telemetry_cleaned

# hulkenberg_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_2_end, end=sector_3_end)
# hulkenberg_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector3_telemetry, hulkenberg)
# hulkenberg_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7908,HUL,01:16:03.734,00:17:12.711,11228.127213,68.448232,6,100.000000,False,21028.940510,859.000000,2115.000000,-239.000000
7909,HUL,01:16:03.997,00:17:12.974,11472.000000,69.444444,6,100.000000,False,21047.310833,992.027640,2129.675521,-238.573770
7910,HUL,01:16:04.154,00:17:13.131,11541.777749,70.292438,6,100.000000,False,21058.459300,1067.000000,2138.000000,-239.000000
7911,HUL,01:16:04.314,00:17:13.291,11612.888860,71.156635,6,100.000000,False,21069.923229,1180.000000,2151.000000,-240.000000
7912,HUL,01:16:04.357,00:17:13.334,11632.000000,71.388889,6,100.000000,False,21073.010833,1210.585293,2154.399724,-239.920333
7913,HUL,01:16:04.493,00:17:13.470,11792.933333,71.861111,6,100.000000,False,21082.794954,1308.000000,2165.000000,-239.000000
7914,HUL,01:16:04.597,00:17:13.574,11916.000000,72.222222,6,100.000000,False,21090.344167,1392.639167,2174.490144,-238.864501
7915,HUL,01:16:04.854,00:17:13.831,10986.210946,73.751984,6,100.000000,False,21109.328752,1614.000000,2200.000000,-240.000000
7916,HUL,01:16:04.877,00:17:13.854,10903.000000,73.888889,7,100.000000,False,21111.033056,1631.364863,2202.009297,-240.074407
7917,HUL,01:16:05.037,00:17:14.014,10548.000000,74.166667,7,100.000000,False,21122.899722,1738.898573,2214.471194,-240.384055


In [18]:
# Q2 data for Kevin Magnussen, Haas
q1, q2, q3 = session.get_laps(magnussen).split_qualifying_sessions()
magnussen_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:55:34.175000"
sector_1_end = "0 days 00:55:51.220000"
sector_2_end = "0 days 00:56:27.265000"
sector_3_end = "0 days 00:56:57.907000"

magnussen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_start, end=sector_1_end)
magnussen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector1_telemetry, magnussen)
magnussen_sector1_telemetry_cleaned

# magnussen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_end, end=sector_2_end)
# magnussen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector2_telemetry, magnussen)
# magnussen_sector2_telemetry_cleaned

# magnussen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_2_end, end=sector_3_end)
# magnussen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector3_telemetry, magnussen)
# magnussen_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
9573,MAG,00:55:34.333,00:21:09.879,11216.850000,68.013889,6,100.000000,False,21050.996494,821.000000,2110.000000,-239.000000
9574,MAG,00:55:34.473,00:21:10.019,11332.000000,68.888889,6,100.000000,False,21060.715556,915.760034,2121.341486,-239.000456
9575,MAG,00:55:34.494,00:21:10.040,11353.699934,68.961805,6,100.000000,False,21062.174312,930.000000,2123.000000,-239.000000
9576,MAG,00:55:34.694,00:21:10.240,11560.366601,69.656250,6,100.000000,False,21076.112894,1067.000000,2138.000000,-239.000000
9577,MAG,00:55:34.713,00:21:10.259,11580.000000,69.722222,6,100.000000,False,21077.448889,1080.341122,2139.489799,-239.002638
9578,MAG,00:55:34.953,00:21:10.499,11667.000000,71.111111,6,100.000000,False,21094.515556,1249.336320,2158.925830,-239.000809
9579,MAG,00:55:34.954,00:21:10.500,11668.114929,71.115278,6,100.000000,False,21094.587126,1250.000000,2159.000000,-239.000000
9580,MAG,00:55:35.134,00:21:10.680,11868.814929,71.865278,6,100.000000,False,21107.528017,1378.000000,2173.000000,-239.000000
9581,MAG,00:55:35.153,00:21:10.699,11890.000000,71.944444,6,100.000000,False,21108.904444,1396.507098,2175.062850,-239.088244
9582,MAG,00:55:35.394,00:21:10.940,11101.585924,73.139881,6,100.000000,False,21126.555639,1668.000000,2206.000000,-241.000000


In [19]:
# Q2 data for Liam Lawson, Visa CashApp
q1, q2, q3 = session.get_laps(lawson).split_qualifying_sessions()
lawson_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:38.695000"
sector_1_end = "0 days 00:57:55.568000"
sector_2_end = "0 days 00:58:31.626000"
sector_3_end = "0 days 00:59:01.907000"

lawson_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_start, end=sector_1_end)
lawson_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector1_telemetry, lawson)
lawson_sector1_telemetry_cleaned

# lawson_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_end, end=sector_2_end)
# lawson_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector2_telemetry, lawson)
# lawson_sector2_telemetry_cleaned

# lawson_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_2_end, end=sector_3_end)
# lawson_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector3_telemetry, lawson)
# lawson_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10335,LAW,00:57:39.394,00:22:54.075,11407.000000,71.111111,6,100.000000,False,21024.762500,1348.549916,2170.331076,-238.782776
10336,LAW,00:57:39.434,00:22:54.115,11437.250000,71.319444,6,100.000000,False,21027.637764,1381.000000,2174.000000,-239.000000
10337,LAW,00:57:39.554,00:22:54.235,11528.000000,71.944444,6,100.000000,False,21036.273611,1479.140000,2185.115878,-239.806108
10338,LAW,00:57:39.774,00:22:54.455,10990.500000,72.777778,6,100.000000,False,21052.357905,1655.000000,2205.000000,-241.000000
10339,LAW,00:57:39.994,00:22:54.675,10453.000000,73.611111,7,100.000000,False,21068.662500,1811.635788,2222.443984,-240.095660
10340,LAW,00:57:40.174,00:22:54.855,10602.500000,74.166667,7,100.000000,False,21082.075068,1934.000000,2236.000000,-239.000000
10341,LAW,00:57:40.354,00:22:55.035,10752.000000,74.722222,7,100.000000,False,21095.562500,2057.926992,2249.786326,-238.537825
10342,LAW,00:57:40.593,00:22:55.274,10788.000000,75.555556,7,100.000000,False,21113.620278,2218.803052,2267.903083,-238.756079
10343,LAW,00:57:40.694,00:22:55.375,10849.722183,75.945216,7,100.000000,False,21121.333501,2280.000000,2275.000000,-239.000000
10344,LAW,00:57:40.953,00:22:55.634,11008.000000,76.944444,7,100.000000,False,21141.320278,2423.713222,2292.017519,-239.880971


In [20]:
# Q2 data for Yuki Tsunoda, Visa CashApp
q1, q2, q3 = session.get_laps(tsunoda).split_qualifying_sessions()
tsunoda_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:26.587000"
sector_1_end = "0 days 00:57:43.456000"
sector_2_end = "0 days 00:58:19.489000"
sector_3_end = "0 days 00:58:49.753000"

tsunoda_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_start, end=sector_1_end)
tsunoda_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector1_telemetry, tsunoda)
tsunoda_sector1_telemetry_cleaned

# tsunoda_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_end, end=sector_2_end)
# tsunoda_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector2_telemetry, tsunoda)
# tsunoda_sector2_telemetry_cleaned

# tsunoda_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_2_end, end=sector_3_end)
# tsunoda_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector3_telemetry, tsunoda)
# tsunoda_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10359,TSU,00:57:26.634,00:22:56.817,11017.200000,68.305556,6,100.000000,False,20977.891596,840.000000,2113.000000,-240.000000
10360,TSU,00:57:26.674,00:22:56.857,11059.000000,68.611111,6,100.000000,False,20980.661389,867.112596,2116.005377,-239.889360
10361,TSU,00:57:26.794,00:22:56.977,11131.301236,69.029521,6,100.000000,False,20988.971344,949.000000,2125.000000,-239.000000
10362,TSU,00:57:26.913,00:22:57.096,11203.000000,69.444444,6,100.000000,False,20997.258611,1031.132933,2133.919451,-238.663011
10363,TSU,00:57:27.073,00:22:57.256,11203.000000,70.555556,6,100.000000,False,21008.547500,1142.977073,2146.125350,-238.929711
10364,TSU,00:57:27.134,00:22:57.317,11262.910651,70.858135,6,100.000000,False,21012.898615,1186.000000,2151.000000,-239.000000
10365,TSU,00:57:27.314,00:22:57.497,11439.696366,71.750992,6,100.000000,False,21025.868412,1314.000000,2166.000000,-239.000000
10366,TSU,00:57:27.353,00:22:57.536,11478.000000,71.944444,6,100.000000,False,21028.691944,1341.832104,2169.273793,-238.948122
10367,TSU,00:57:27.494,00:22:57.677,11591.504948,72.531944,6,100.000000,False,21038.933559,1443.000000,2181.000000,-239.000000
10368,TSU,00:57:27.553,00:22:57.736,11639.000000,72.777778,6,100.000000,False,21043.247500,1485.893558,2185.784863,-239.480049


In [21]:
# Q3 data for George Russell, Mercedes
q1, q2, q3 = session.get_laps(russell).split_qualifying_sessions()
russell_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:17:14.677000"
sector_1_end = "0 days 01:17:31.522000"
sector_2_end = "0 days 01:18:07.527000"
sector_3_end = "0 days 01:18:37.566000"

russell_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_start, end=sector_1_end)
russell_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector1_telemetry, russell)
russell_sector1_telemetry_cleaned

# russell_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_end, end=sector_2_end)
# russell_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector2_telemetry, russell)
# russell_sector2_telemetry_cleaned

# russell_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_2_end, end=sector_3_end)
# russell_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector3_telemetry, russell)
# russell_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8155,RUS,01:17:14.754,00:17:43.740,11594.541120,68.036159,6,100.000000,False,20972.506329,911.000000,2121.000000,-239.000000
8156,RUS,01:17:14.958,00:17:43.944,11767.000000,69.166667,6,100.000000,False,20986.695556,1060.828172,2137.641895,-239.053742
8157,RUS,01:17:15.198,00:17:44.184,10417.000000,70.277778,6,100.000000,False,21003.562222,1210.711273,2154.077438,-239.015342
8158,RUS,01:17:15.213,00:17:44.199,10429.833388,70.358797,6,100.000000,False,21004.629670,1219.000000,2155.000000,-239.000000
8159,RUS,01:17:15.558,00:17:44.544,10725.000000,72.222222,7,100.000000,False,21029.562222,1395.654483,2175.252084,-238.817847
8160,RUS,01:17:15.594,00:17:44.580,10741.560000,72.322222,7,100.000000,False,21032.175524,1419.000000,2178.000000,-239.000000
8161,RUS,01:17:15.734,00:17:44.720,10805.960000,72.711111,7,100.000000,False,21042.359768,1521.000000,2190.000000,-240.000000
8162,RUS,01:17:15.758,00:17:44.744,10817.000000,72.777778,7,100.000000,False,21044.117778,1538.560799,2192.022428,-240.104807
8163,RUS,01:17:15.954,00:17:44.940,10931.927273,73.643939,7,100.000000,False,21058.636884,1682.000000,2208.000000,-240.000000
8164,RUS,01:17:16.198,00:17:45.184,11075.000000,74.722222,7,100.000000,False,21076.995556,1885.756657,2230.326298,-239.001758


Sector 1 telemetry data for all drivers is concatanated into a single dataframe below.

In [22]:
total_sector1_telemetry = pd.concat([
    norris_sector1_telemetry_cleaned, 
    piastri_sector1_telemetry_cleaned, 
    verstappen_sector1_telemetry_cleaned, 
    perez_sector1_telemetry_cleaned,
    sainz_sector1_telemetry_cleaned,
    leclerc_sector1_telemetry_cleaned,
    bottas_sector1_telemetry_cleaned,
    alonso_sector1_telemetry_cleaned,
    stroll_sector1_telemetry_cleaned,
    gasly_sector1_telemetry_cleaned,
    hulkenberg_sector1_telemetry_cleaned,
    magnussen_sector1_telemetry_cleaned,
    lawson_sector1_telemetry_cleaned,
    tsunoda_sector1_telemetry_cleaned,
    russell_sector1_telemetry_cleaned
], axis=0)

total_sector1_telemetry

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7979,NOR,01:16:59.557,00:17:19.828,11229.000000,68.333333,6,100.000000,False,20968.508889,778.569904,2105.751713,-239.001361
7980,NOR,01:16:59.654,00:17:19.925,11318.724941,68.932098,6,100.000000,False,20975.275273,844.000000,2113.000000,-239.000000
7981,NOR,01:16:59.874,00:17:20.145,11522.224941,70.290123,6,100.000000,False,20990.849977,997.000000,2130.000000,-239.000000
7982,NOR,01:16:59.917,00:17:20.188,11562.000000,70.555556,6,100.000000,False,20993.908889,1026.891384,2133.312099,-238.974188
7983,NOR,01:17:00.055,00:17:20.326,11665.883237,70.981481,6,100.000000,False,21003.748877,1123.000000,2144.000000,-239.000000
7984,NOR,01:17:00.274,00:17:20.545,11830.741618,71.657407,6,100.000000,False,21019.491494,1280.000000,2162.000000,-240.000000
7985,NOR,01:17:00.277,00:17:20.548,11833.000000,71.666667,6,100.000000,False,21019.708889,1282.167908,2162.253093,-240.006256
7986,NOR,01:17:00.474,00:17:20.745,11091.172116,72.692708,6,100.000000,False,21034.089617,1425.000000,2179.000000,-240.000000
7987,NOR,01:17:00.597,00:17:20.868,10628.000000,73.333333,7,100.000000,False,21043.175556,1514.747481,2189.147597,-240.558748
7988,NOR,01:17:00.693,00:17:20.964,10657.866667,73.777778,7,100.000000,False,21050.324912,1585.000000,2197.000000,-241.000000


Sector 2 telemetry data for all drivers is concatenated into a single dataframe below.

In [ ]:
total_sector2_telemetry = pd.concat([
    norris_sector2_telemetry_cleaned, 
    piastri_sector2_telemetry_cleaned, 
    verstappen_sector2_telemetry_cleaned, 
    perez_sector2_telemetry_cleaned,
    sainz_sector2_telemetry_cleaned,
    leclerc_sector2_telemetry_cleaned,
    bottas_sector2_telemetry_cleaned,
    alonso_sector2_telemetry_cleaned,
    stroll_sector2_telemetry_cleaned,
    gasly_sector2_telemetry_cleaned,
    hulkenberg_sector2_telemetry_cleaned,
    magnussen_sector2_telemetry_cleaned,
    lawson_sector2_telemetry_cleaned,
    tsunoda_sector2_telemetry_cleaned,
    russell_sector2_telemetry_cleaned
], axis=0)

total_sector2_telemetry

Sector 3 telemetry data for all drivers is concatanated into a single dataframe below.

In [ ]:
total_sector3_telemetry = pd.concat([
    norris_sector3_telemetry_cleaned, 
    piastri_sector3_telemetry_cleaned, 
    verstappen_sector3_telemetry_cleaned, 
    perez_sector3_telemetry_cleaned,
    sainz_sector3_telemetry_cleaned,
    leclerc_sector3_telemetry_cleaned,
    bottas_sector3_telemetry_cleaned,
    alonso_sector3_telemetry_cleaned,
    stroll_sector3_telemetry_cleaned,
    gasly_sector3_telemetry_cleaned,
    hulkenberg_sector3_telemetry_cleaned,
    magnussen_sector3_telemetry_cleaned,
    lawson_sector3_telemetry_cleaned,
    tsunoda_sector3_telemetry_cleaned,
    russell_sector3_telemetry_cleaned
], axis=0)

total_sector3_telemetry